In [4]:
import pandas as pd
import json
import csv
import re

In [5]:
players = pd.read_csv("./data/players.csv", delimiter="\t")
players = players.drop_duplicates()

In [8]:
players['BirthCountry'] = None
players['Age'] = None
players['Role'] = None

In [9]:
age_pattern = r'\(age (\d+)\)'

for index, player in players.iterrows():
    country_of_birth = None
    role = None
    age = None
    for info in player['PlayerCard'].split(';'):
        if 'Country of Birth' in info:
            country_of_birth = info.replace('Country of Birth', '').strip()
        if 'Role' in info:
            role = info.replace('Role', '').strip()
            
        match = re.search(age_pattern, info)
        # Extract the age if a match is found
        if match:
            age = match.group(1)
            
    player['BirthCountry'] = country_of_birth
    player['Age'] = age
    player['Role'] = role

In [60]:
players['player_id'] = [i for i in range(len(players))]

In [38]:
teams_df = pd.DataFrame(columns=['player_id', 'team', 'since', 'till'])
team_name_pattern = r'Team (.+?) -'
since_pattern = r'Since: (\d{4}-\d{2}-\d{2})'
till_pattern = r'Till: (\w+)'
i = 10

for index, player in players.iterrows():
    if pd.isnull(player.TeamHistory):
        continue
    for team_info in player.TeamHistory.split(';'):
        if team_info == '':
            continue
        # Extract information using re.search
        team_name_match = re.search(team_name_pattern, team_info)
        since_match = re.search(since_pattern, team_info)
        till_match = re.search(till_pattern, team_info)

        # Get the extracted information
        team_name = team_name_match.group(1) if team_name_match else None
        since_date = since_match.group(1) if since_match else None
        till_info = till_match.group(1) if till_match else None

        # If "Till" is "Present", set till_date to today's date; otherwise, use the extracted value
        if pd.isnull(till_info) or till_info.lower() == "present":
            till_date = datetime.now().strftime("%Y-%m-%d")
        else:
            till_date = till_info

        # Print the information
        if team_name is not None:
            teams_df = pd.concat([teams_df, pd.DataFrame({'player_id':[player.id], 'team' : [team_name], 'since': [since_date], 'till': [till_date]})], ignore_index=True)
            new_row = {'player_id': '1', 'team': 'Team A', 'since': '2022-01-01', 'till': '2022-12-31'}

In [41]:
teams_df.to_csv('data/player_teams.csv', sep='\t', index=False)
players.to_csv('data/players_processed.csv', sep='\t', index=False)

In [47]:
unique_teams = list(teams_df.team.unique())

In [48]:
with open('data/teams_list.txt', 'w') as f: 
    for t in unique_teams:
        f.write(str(t)+"\n")

In [57]:
with open('data/countries_list.txt', 'w') as f:
    for c in list(players.BirthCountry.unique()):
        f.write(str(c)+"\n")

In [65]:
players = players.drop('id', axis=1)

In [ ]:
teams_df

In [66]:
merged_df = pd.merge(teams_df, players, on='player_id', how='outer')

/tmp/ipykernel_213734/2051207727.py:1: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  merged_df = pd.merge(teams_df, players, on='player_id', how='outer')


In [68]:
players

,PlayerName,TeamHistory,HistoryText,PlayerCard,TournamentResults,ChampionsStats,BirthCountry,Age,Role,player_id
0,Shim Sung-bo,NaN,NaN,Shim Sung-bo ; ; Background Information ; Name...,NaN,NaN,South Korea,None,None,0
1,LastSurvivor,"Team Nordis Team - Region: EU , Lane: Coach , ...","EU ⁠ ⁠ NOR January 16, Szafa , Wiko , Jhein , ...",Retired ; LastSurvivor ; ; Background Informat...,LastSurvivor Tournament Results; Date Pl Event...,NaN,Poland,24,Coach,1
2,LIMIT (Dino Tot),"Team - Region: EU , Lane: Support , Since: ,...","EU ⁠ ⁠ BONT November (approx.), Darkrai , Bage...",LIMIT ; ; Background Information ; Name Dino T...,LIMIT (Dino Tot) Tournament Results; Date Pl E...,Player: LIMIT (Dino Tot) - Showing Values Per ...,Croatia,25,Support,2
3,Noway (Nguyễn Vũ Long),"Team - Region: SEA , Lane: Bot Laner , Since:...","SEA ⁠ ⁠ SR January (approx.), Noway joins. e ;...",Noway ; ; Background Information ; Name Nguyễn...,Noway (Nguyễn Vũ Long) Tournament Results; Dat...,Player: Noway (Nguyễn Vũ Long) - Showing Value...,Vietnam,26,Bot Laner,3
5,Sawyor,"Team Gunrunners - Region: EU , Lane: Jungler ,...","EU ⁠ ⁠ GRS June 9, Sawyor and Soul join. Thrak...",Sawyor ; ; Background Information ; Name Jiří ...,Sawyor Tournament Results; Date Pl Event Last ...,Player: Sawyor - Showing Values Per Game - Ope...,Czech Republic,23,Jungler,4
...,...,...,...,...,...,...,...,...,...,...
14650,Nightfaller,"Team UQ Union - Region: OCE , Lane: Top Laner ...","OCE ⁠ ⁠ UQU February 26, Nightfaller , Alani ,...",Retired ; Nightfaller ; ; Background Informati...,Nightfaller Tournament Results; Date Pl Event ...,Player: Nightfaller - Showing Values Per Game ...,Australia,None,Top Laner,13099
14651,Candy Man,"Team - Region: OCE , Lane: Jungler , Since: 2...","OCE ⁠ ⁠ DS June 27, Framer and Cappy join. Rav...",Retired ; Candy Man ; ; Background Information...,Candy Man Tournament Results; Date Pl Event La...,Player: Candy Man - Showing Values Per Game - ...,Australia,None,Jungler,13100
14652,Kisee,"Team UQ Union - Region: OCE , Lane: Mid Laner ...","OCE ⁠ ⁠ UQU April 23, Cappy , Lansen , Ryot , ...",Kisee ; ; Background Information ; Name Ronald...,NaN,NaN,Australia,20,Mid Laner,13101
14653,Lansen,"Team UQ Union - Region: OCE , Lane: Mid Laner ...","OCE ⁠ ⁠ UQU April 23, Cappy , Lansen , Ryot , ...",Retired ; Lansen ; ; Background Information ; ...,Lansen Tournament Results; Date Pl Event Last ...,Player: Lansen - Showing Values Per Game - Ope...,Australia,22,Mid Laner,13102


In [73]:
merged_df[merged_df.BirthCountry == 'Slovakia'][merged_df.team == 'eSuba'][merged_df.since > '2020']

/tmp/ipykernel_213734/2792936205.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  merged_df[merged_df.BirthCountry == 'Slovakia'][merged_df.team == 'eSuba'][merged_df.since > '2020']


,player_id,team,since,till,PlayerName,TeamHistory,HistoryText,PlayerCard,TournamentResults,ChampionsStats,BirthCountry,Age,Role
11815,2065.0,eSuba,2020-06-16,2020,Wondro,"Team Dark Tigers - Region: EU , Lane: Top Lane...","EU ⁠ ⁠ DTG February 11, Wondro , DyBer , Skot ...",Wondro ; ; Background Information ; Name Ondre...,Wondro Tournament Results; Date Pl Event Last ...,Player: Wondro - Showing Values Per Game - Ope...,Slovakia,None,Top Laner
